Execute the following block of code by selecting it and clicking ``ctrl + enter`` to create an ``NvidiaRacecar`` class.  

In [8]:
# Complete working NvidiaRacecar for the training notebook
import traitlets
import smbus
import time

class Racecar(traitlets.HasTraits):
    steering = traitlets.Float()
    throttle = traitlets.Float()
    
    @traitlets.validate('steering')
    def _clip_steering(self, proposal):
        return max(-1.0, min(1.0, proposal['value']))
        
    @traitlets.validate('throttle')
    def _clip_throttle(self, proposal):
        return max(-1.0, min(1.0, proposal['value']))

class NvidiaRacecar(Racecar):
    # These are the attributes the notebook expects
    i2c_address = traitlets.Integer(default_value=0x40)
    steering_gain = traitlets.Float(default_value=-0.65)
    steering_offset = traitlets.Float(default_value=0)
    steering_channel = traitlets.Integer(default_value=0)
    throttle_gain = traitlets.Float(default_value=0.8)
    throttle_channel = traitlets.Integer(default_value=1)
    
    def __init__(self, *args, **kwargs):
        super(NvidiaRacecar, self).__init__(*args, **kwargs)
        self.bus = smbus.SMBus(7)
        self.center_pulse = 1500
        self._initialize_pca9685()
        print("✅ NvidiaRacecar ready for training!")
    
    def _initialize_pca9685(self):
        try:
            self.bus.write_byte_data(self.i2c_address, 0x00, 0x10)
            time.sleep(0.005)
            prescale = int(25000000 / (4096 * 50) - 1)
            self.bus.write_byte_data(self.i2c_address, 0xFE, prescale)
            self.bus.write_byte_data(self.i2c_address, 0x00, 0x20)
            time.sleep(0.005)
            self.bus.write_byte_data(self.i2c_address, 0x01, 0x04)
            self._set_servo_pulse(0, self.center_pulse)
            self._set_servo_pulse(1, self.center_pulse)
        except Exception as e:
            print(f"Warning: {e}")
    
    def _set_servo_pulse(self, channel, pulse_us):
        pulse_us = max(1000, min(2000, pulse_us))
        pwm_value = int((pulse_us * 4096) / 20000)
        base_reg = 0x06 + 4 * channel
        try:
            self.bus.write_byte_data(self.i2c_address, base_reg, 0)
            self.bus.write_byte_data(self.i2c_address, base_reg + 1, 0)
            self.bus.write_byte_data(self.i2c_address, base_reg + 2, pwm_value & 0xFF)
            self.bus.write_byte_data(self.i2c_address, base_reg + 3, (pwm_value >> 8) & 0xFF)
        except Exception as e:
            print(f"PWM Error: {e}")
    
    @traitlets.observe('steering')
    def _on_steering(self, change):
        scaled = change['new'] * self.steering_gain + self.steering_offset
        pulse_us = self.center_pulse + (scaled * 500)
        self._set_servo_pulse(self.steering_channel, pulse_us)
    
    @traitlets.observe('throttle')
    def _on_throttle(self, change):
        scaled = change['new'] * self.throttle_gain
        if scaled > 0:
            pulse_us = self.center_pulse - (scaled * 200)  # Forward for your ESC
        elif scaled < 0:
            pulse_us = self.center_pulse + (abs(scaled) * 200)  # Reverse
        else:
            pulse_us = self.center_pulse
        self._set_servo_pulse(self.throttle_channel, pulse_us)

# Create the car object the notebook expects
car = NvidiaRacecar()

# Test that it has the right attributes
print(f"✅ steering_gain: {car.steering_gain}")
print(f"✅ steering_offset: {car.steering_offset}") 
print(f"✅ throttle_gain: {car.throttle_gain}")
print("🎯 Ready for AI training!")

✅ NvidiaRacecar ready for training!
✅ steering_gain: -0.65
✅ steering_offset: 0.0
✅ throttle_gain: 0.8
🎯 Ready for AI training!


The ``NvidiaRacecar`` implements the ``Racecar`` class, so it has two attributes ``throttle`` and ``steering``. 

We can assign values in the range ``[-1, 1]`` to these attributes.  Execute the following to set the steering to 0.4.

> If the car does not respond, it may still be in ``manual`` mode.  Flip the manual override switch on the RC transmitter.

In [9]:
car.steering = 0


The ``NvidiaRacecar`` class has two values ``steering_gain`` and ``steering_bias`` that can be used to calibrate the steering.

We can view the default values by executing the cells below.

In [11]:
print(car.steering_gain)

-0.65


In [12]:
print(car.steering_offset)

0.0


The final steering value is computed using the equation

$y = a \times x + b$

Where,

* $a$ is ``car.steering_gain``
* $b$ is ``car.steering_offset``
* $x$ is ``car.steering``
* $y$ is the value written to the motor driver

You can adjust these values calibrate the car so that setting a value of ``0`` moves forward, and setting a value of ``1`` goes fully right, and ``-1`` fully left.

To set the throttle of the car to ``0.2``, you can call the following.

> Give JetRacer lots of space to move, and be ready on the manual override, JetRacer is *fast*

In [17]:
car.throttle = 0

The throttle also has a gain value that could be used to control the speed response.  The throttle output is computed as

$y = a \times x$

Where,

* $a$ is ``car.throttle_gain``
* $x$ is ``car.throttle``
* $y$ is the value written to the speed controller

Execute the following to print the default gain

In [18]:
print(car.throttle_gain)

0.8


Set the following to limit the throttle to half

In [19]:
car.throttle_gain = 0.5

Please note the throttle is directly mapped to the RC car.  When the car is stopped and a negative throttle is set, it will reverse.  If the car is moving forward and a negative throttle is set, it will brake.

That's it for this notebook!